# Урок 2. Создание реляционной базы данных

## Домашнее задание

### Базовый уровень
1. Загрузить файлы в созданную базу данных

2. Выслать результаты запроса подсчёта пациентов по полу (`SELECT sex_concept_id, COUNT(*) FROM gb_rwd.person GROUP BY sex_concept_id`) в виде файла с названием "lesson2_{your_surname}.csv"

#### Создание БД
На первом этапе, необходимо создать базу данных (можно использовать дефолтную)

Затем, в этой базе данных создадим схему gb_rwd


В каждой таблице есть первичный ключ (который мы не обозначаем явным образом), который позволяет объединять все таблицы в случае написания запроса


Создадим свою базу данных для RWD исследования
```
CREATE schema gb_rwd;

CREATE TABLE gb_rwd.person (
  person_id int, -- первичный ключ
  date_of_birth date,
  sex_concept_id int, -- вторичный ключ
  hospital_days int
);

CREATE TABLE gb_rwd.measurement (
  measurement_id int, -- первичный ключ
  measurement_concept_id int, -- вторичный ключ
  measurement_date date,
  value float,
  person_id int -- вторичный ключ
);

CREATE TABLE gb_rwd.vocabulary (
  concept_id int, -- первичный ключ
  concept_name varchar(255),
  domain varchar(150),
  code varchar(150)
);

CREATE TABLE gb_rwd.condition (
  condition_id int, -- первичный ключ
  condition_concept_id int, -- вторичный ключ
	condition_date date,
  person_id int -- вторичный ключ
);

CREATE TABLE gb_rwd.drug (
  drug_id int, -- первичный ключ
  drug_concept_id int, -- вторичный ключ
  drug_date date,
  person_id int -- вторичный ключ
);

CREATE TABLE gb_rwd.procedure (
  procedure_id int, -- первичный ключ
  procedure_concept_id int, -- вторичный ключ
  procedure_date date,
  person_id int -- вторичный ключ
);

```



База данные в excel формате
Скачайте и вставьте таблицу словарь в sql таблицу vocabulary

Например, если мы хотим выбрать все значения пациентов возраста от 18 до 65 лет.

Для этого сначала выбираем записи из таблицы `person`, затем, соединяем с таблицей `measurement` по полю `person_id`

## Подготовка csv для базы данных

In [1]:
import warnings
warnings.filterwarnings("ignore")
import re
from re import sub, search, findall
import pandas  as pd
from datetime import datetime
import numpy as np
import glob
import os

In [2]:
!chcp 65001

Active code page: 65001


In [3]:
def epi_dates_preparation(strng):
    data_1 =  sub(r'(0?[1-9]|[12]\d|30|31)[.](0?[1-9]|1[0-2])[.](\d{4})',
                     '\\1-\\2-\\3', strng)
    return sub(r'(0?[1-9]|[12]\d|30|31)[.](0?[1-9]|1[0-2])[.](\d{2})',
                  '\\1-\\2-20\\3', data_1)

Примитивный, но работоспособный вариант определения пола: удаляем всё, что есть в эпикризе после 1 упоминания слова "диагноз" и ищем "вна", а у мужчин - "вич"

In [4]:
def read_txt(ff):
  with open(ff, encoding='utf-8', errors='ignore') as f:
    f = f.read()
    return f

In [5]:
def get_gender(file_):
  file_ = sub('диагноз.*', '', file_)
  if 'вна' in file_:
    return 25
  else:
    return 26


Убираем "лишнеее".
Подходы: фичи или мусор?

In [6]:
def remover(file_):
  file_ = ''.join(file_.split()).lower()
  stop_element = [',',':', '/t', 'менее']
  for elem in stop_element:
    if elem == ',':
      file_ = sub(elem, '.', file_)
    else:
      file_ = sub(elem, '', file_)
  return(file_)

``` def preparator(path):
	def inner_preparator(fun):
		def wrapper(*args, **kwargs):
			txt2 = epi_dates_preparation(remover(read_txt(path)))
			return fun(txt2)
		return wrapper
	return inner_preparator ```



```
CREATE TABLE gb_rwd.person (
  person_id int, -- первичный ключ
  date_of_birth date,
  sex_concept_id int, -- вторичный ключ
  hospital_days int
);
```



Собираем все файлы

In [8]:
lst_of_txts = list(glob.glob(os.path.join('22_2_SEM_files/rwd_raw', '*.txt')))
lst_of_txts

['22_2_SEM_files/rwd_raw\\file (1067).txt',
 '22_2_SEM_files/rwd_raw\\file (1068).txt',
 '22_2_SEM_files/rwd_raw\\file (1069).txt',
 '22_2_SEM_files/rwd_raw\\file (107).txt',
 '22_2_SEM_files/rwd_raw\\file (1070).txt',
 '22_2_SEM_files/rwd_raw\\file (1071).txt',
 '22_2_SEM_files/rwd_raw\\file (1072).txt',
 '22_2_SEM_files/rwd_raw\\file (1073).txt',
 '22_2_SEM_files/rwd_raw\\file (1074).txt',
 '22_2_SEM_files/rwd_raw\\file (1075).txt',
 '22_2_SEM_files/rwd_raw\\file (1078).txt',
 '22_2_SEM_files/rwd_raw\\file (1079).txt',
 '22_2_SEM_files/rwd_raw\\file (108).txt',
 '22_2_SEM_files/rwd_raw\\file (1080).txt',
 '22_2_SEM_files/rwd_raw\\file (1081).txt',
 '22_2_SEM_files/rwd_raw\\file (1082).txt',
 '22_2_SEM_files/rwd_raw\\file (1083).txt',
 '22_2_SEM_files/rwd_raw\\file (1084).txt',
 '22_2_SEM_files/rwd_raw\\file (1085).txt',
 '22_2_SEM_files/rwd_raw\\file (1086).txt',
 '22_2_SEM_files/rwd_raw\\file (1087).txt',
 '22_2_SEM_files/rwd_raw\\file (1088).txt',
 '22_2_SEM_files/rwd_raw\\file (10

In [9]:
len(lst_of_txts)

34

*Функциональный подход*

In [10]:
patient_ids = list(map(lambda file_: int(search(r'\d+', file_)[0]), lst_of_txts))
list_of_epi = list(map(lambda file_: epi_dates_preparation(remover(read_txt(file_))) , lst_of_txts))
gender_list = list(map(lambda file_: get_gender(epi_dates_preparation(remover(read_txt(file_)))) , lst_of_txts))

In [11]:
gender_list

[25,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 26,
 26,
 26,
 25,
 26,
 25,
 25,
 26,
 26,
 25,
 26,
 25,
 26,
 25,
 25,
 26,
 25,
 25,
 25,
 25,
 26,
 26,
 26,
 25,
 25,
 25]

### Начнём с таблицы `person`

In [14]:
def prepare_person_table(
    gender_list,
    list_of_epi,
    patient_ids,
    hospital_days = [],
    date_of_birth = []
    ):
    for file_ in list_of_epi:
        admission = findall('\d{2}-\d{2}-\d{4}', file_)[1]
        discharge = findall('\d{2}-\d{2}-\d{4}', file_)[-1]
        hospital_days.append(
            datetime.strptime(discharge, '%d-%m-%Y').date() -
            datetime.strptime(admission, '%d-%m-%Y').date())
        date_of_birth.append(findall('\d{2}-\d{2}-\d{4}', file_)[0])
        data_dct = {'person_id ': patient_ids, 'date_of_birth': date_of_birth, 'hospital_days':hospital_days,'sex_concept_id ': gender_list}
        df = pd.DataFrame(data_dct)
        df = df.replace(r'', np.nan, regex=True)
        df['hospital_days'] = df['hospital_days'].dt.days.astype('int16')
        df['date_of_birth'] = pd.to_datetime(df.date_of_birth)
        return df

In [15]:
patient_table = prepare_person_table(gender_list = gender_list,
                          list_of_epi = list_of_epi,
                          patient_ids = patient_ids)
patient_table

ValueError: All arrays must be of the same length

In [17]:
patient_table.to_csv('person.csv', index=False)

### Создание файла для таблицы `measurement`



```
CREATE TABLE gb_rwd.measurement (
  measurement_id int, -- первичный ключ
  measurement_concept_id int, -- вторичный ключ
  measurement_date date,
  value float,
  person_id int -- вторичный ключ
);
```



Для того, чтобы сделать процесс формирования таблицы более программатичным,
создадим словарь где ключ - название анализа, а значение это словарь в котором ключ тип исследования, а значение это идентификатор

In [19]:
measurements_map = {
    'фибриноген': {'гемостазиограмма': 6},
    'креатинин': {'биохимическоеисследованиекрови': 7},
    'лейкоциты': {'общийанализкрови': 8},
    'гемоглобин': {'общийанализкрови': 9},
    'тромбоциты': {'общийанализкрови': 10},
    '.-реактивныйбелок': {'биохимическоеисследованиекрови': 11},
    'соэ': {'общийанализкрови': 12},
    'лимфоциты': {'общийанализкрови': 13}
    ,'прокальцитонин': {'ифаанализ': 27}
}

In [20]:
for measurement_name , sub_dict in measurements_map.items():
    for measurement_type, measurement_concept_id_ in sub_dict.items():
      print(sub_dict, measurement_concept_id_)

{'гемостазиограмма': 6} 6
{'биохимическоеисследованиекрови': 7} 7
{'общийанализкрови': 8} 8
{'общийанализкрови': 9} 9
{'общийанализкрови': 10} 10
{'биохимическоеисследованиекрови': 11} 11
{'общийанализкрови': 12} 12
{'общийанализкрови': 13} 13
{'ифаанализ': 27} 27


In [21]:
def measurements_template(
    list_of_epicrisis,
    patient_ids,
    measurements_map,
    measurement_id = [],
    measurement_date  = [],
    measurement_concept_id = [],
    patient_id = [],
    value = []
    ):
  for measurement_name , sub_dict in measurements_map.items():
    for measurement_type, concept_id in sub_dict.items():
        for file_, patient_id_ in zip(list_of_epicrisis, patient_ids):
            pattern0 = ''.join(['r(\d{2}-\d{2}-\d{4})(', measurement_type, ')'])
            file_ = sub(pattern0, r'\2\1', file_)
            pattern1 = ''.join(['(?<=', measurement_type, ')', '(\d{2}-\d{2}-\d{4}|)'])
            for date in findall(pattern1, file_):
              measurement_date .append(date)
              patient_id.append(patient_id_)
              measurement_concept_id.append(concept_id)
            pattern2 = ''.join(['(', measurement_type, ')', '(\d{2}-\d{2}-\d{4}|)'])
            file_1 = sub(pattern2, r'\2\1', file_)
            pattern3 = ''.join(['(', measurement_type, '.*?)', '(?=\d{2}-\d{2}-\d{4})'])
            for value_ in findall(pattern3, file_1):
              value_ = sub(''.join(['(.*)(', measurement_name, ')', '(\d.\d*)']), r'\3\2\1', value_)
              value_ = sub(''.join(['(', measurement_name, '|', measurement_type, ').*']) , '', value_)
              value_ = re.sub('[^0-9.]|х10|x10', '', value_)

              if value_:
                  value.append(value_)
              else:
                  value.append('NA')

  data_dct = {'person_id': patient_id, 'measurement_concept_id': measurement_concept_id,
                      'measurement_date': measurement_date ,'value': value}
  df = pd.DataFrame.from_dict(data_dct, orient='index')

  df = df.transpose()
  df['measurement_id'] = df.index + 1
  df = df[df.measurement_date != '']
  df = df[df.value != 'NA']
  df['measurement_date'] = pd.to_datetime(df.measurement_date)

  return df

In [22]:
patient_ids = list(map(lambda file_: int(search(r'\d+', file_)[0]), lst_of_txts))
list_of_epi = list(map(lambda file_: epi_dates_preparation(remover(read_txt(file_))) , lst_of_txts))
measurement_data = measurements_template(list_of_epicrisis = list_of_epi, patient_ids = patient_ids, measurements_map = measurements_map)
measurement_data

,person_id,measurement_concept_id,measurement_date,value,measurement_id
0,1067,6,2020-12-14,4.8,1
1,1068,6,2020-12-14,6.38,2
2,1069,6,2020-12-13,5.9,3
3,107,6,2021-01-20,2.42,4
4,107,6,2021-01-21,4,5
...,...,...,...,...,...
1490,1102,27,2020-12-24,None,1491
1491,1103,27,2020-12-21,None,1492
1492,1103,27,2020-12-21,None,1493
1493,1103,27,2020-12-22,None,1494


In [24]:
measurement_data.to_csv('measurements.csv', index=False)

Учитывая, что ошибки в написании препаратов это нередкость и препараты могут иметь разные коммерческие названия, тогда наша задача сделать "маппинг" этих препаратов к ингредиенту.

In [23]:
treatment_map = {
    'лизиноприл':{'л.з.....ил|лизитар|лизинеоприл|лизоретик|диротон': 20},
    'цефтриаксон':{'три.....ф|цефт.....он|цефтриакосн': 16},
    "бисопролол":{"б.с.пр..ол|бисопралдол|бикард|конкор": 21},
     "аспирин": {"аспкиард|кардиомагнил|ас....рд": 22},
     "дексаметазон": {"декс.......н|дексаетазон": 15},
     "азитромицин": {"азит.......": 18},
    'гепарин': {'гепарин': 19 }
}



```
CREATE TABLE gb_rwd.drug (
  drug_id int, -- первичный ключ
  drug_concept_id int, -- вторичный ключ
  drug_date date,
  person_id int -- вторичный ключ
);
```



In [24]:
def treatment_detection(
    list_of_epicrisis,
    patient_ids,
    treatment_map,
    drug_date  = [],
    drug_concept_id = [],
    patient_id = []
    ):
  for drug_name , sub_dct in treatment_map.items():
    for variations, drug_id in    sub_dct.items():
      for file_, patient_id_ in zip(list_of_epicrisis, patient_ids):
        file_ = sub(variations, drug_name, file_)
        if drug_name in  file_:
          drug_date.append(findall('\d{2}-\d{2}-\d{4}', file_)[-1])
          drug_concept_id.append(drug_id)
          patient_id.append(patient_id_)
    data_dct = {'person_id': patient_id, 'drug_concept_id': drug_concept_id,
                      'drug_date': drug_date }
  df = pd.DataFrame(data_dct)
  df['drug_id'] = df.index + 1
  df['drug_date'] = pd.to_datetime(df.drug_date)
  return(df)

In [25]:
drug_data = treatment_detection(
    list_of_epicrisis = list_of_epi,
    patient_ids = patient_ids,
     treatment_map = treatment_map
)

In [30]:
drug_data

,person_id,drug_concept_id,drug_date,drug_id
0,1073,20,2020-12-24,1
1,1079,20,2021-01-12,2
2,1097,20,2021-01-14,3
3,1070,16,2021-02-01,4
4,1072,16,2020-12-18,5
...,...,...,...,...
79,1094,19,2020-12-28,80
80,1096,19,2020-12-30,81
81,1098,19,2021-01-19,82
82,1100,19,2021-01-04,83


In [31]:
drug_data.to_csv('drugs.csv', index=False)

Сейчас поработаем с доменом "Заболевания"

In [26]:
condition_procedures_map = {
    1:	{'Внегоспитальная пневмония': 'пне...ния|внегоспитальная|внебольничная'},
    2:	{'Ишемическая болезнь сердца': 'ссн|ибс|атеросклеротический|кардиосклероз|фп|фибрил....' },
    3:	{'Сахарный диабет': 'д.абет|сдтип|сд2|сд1|сах.....'},
    4:	{'Коронавирусная инфекция': 'covid19|коронавирусная|b34|sarscov2'},
    5:	{'Артериальная гипертензия': 'аг\d|агi|артериальнаягипер '},
    23:	{'Рентгелологическое исследование грудной клетки': 'rgогк|ргогк'},
    24:	{'Электрокардиография': 'экг|электрокардио'}
}

In [27]:
def condition_detection(
    list_of_epicrisis,
    patient_ids,
    condition_map,
    condition_date  = [],
    condition_concept_id = [],
    patient_id = []
    ):
  for concept_id , sub_dct in condition_map.items():
    if concept_id < 6:
      for condition, variations in  sub_dct.items():
        for file_, patient_id_ in zip(list_of_epicrisis, patient_ids):
          file_ = sub(variations, condition, file_)
          if condition in  file_:
            condition_date.append(findall('\d{2}-\d{2}-\d{4}', file_)[-1])
            condition_concept_id.append(concept_id)
            patient_id.append(patient_id_)
    data_dct = {'person_id': patient_id, 'condition_concept_id': condition_concept_id,
                      'condition_date': condition_date }
  df = pd.DataFrame(data_dct)
  df['condition_id'] = df.index + 1
  df['condition_date'] = pd.to_datetime(df.condition_date)
  return(df)

In [28]:
condition_data = condition_detection(
    list_of_epicrisis = list_of_epi,
    patient_ids = patient_ids,
     condition_map = condition_procedures_map
)
condition_data.to_csv('conditions.csv', index=False)

In [29]:
def procedures_detection(
    list_of_epicrisis,
    patient_ids,
    procedure_map,
    procedure_date  = [],
    procedure_concept_id = [],
    patient_id = []
    ):
  for concept_id , sub_dct in procedure_map.items():
    if concept_id > 6:
      for procedure, variations in  sub_dct.items():
        for file_, patient_id_ in zip(list_of_epicrisis, patient_ids):
          file_ = sub(variations, procedure, file_)
          if procedure in  file_:
            procedure_date.append(findall('\d{2}-\d{2}-\d{4}', file_)[-1])
            procedure_concept_id.append(concept_id)
            patient_id.append(patient_id_)
    data_dct = {'person_id': patient_id, 'procedure_concept_id': procedure_concept_id,
                      'procedure_date': procedure_date}
  df = pd.DataFrame(data_dct)
  df['procedure_id'] = df.index + 1
  df['procedure_date'] = pd.to_datetime(df.procedure_date)
  return(df)

In [30]:
procedures_data = procedures_detection(
    list_of_epicrisis = list_of_epi,
    patient_ids = patient_ids,
    procedure_map = condition_procedures_map
)
procedures_data.to_csv('procedures.csv', index=False)

Импортируем данные в нашу базу данных

### Продвинутый уровень

1.   Создать класс EpicPars
2.   Создать документацию для пользователя
3.   Сделать пример использования вашего импортируемого класса


***Необходимо отметить, что написание документации - важнейший навык в работе, т.к. ваши коллеги не имеют тот же опыт, что и вы. Поэтому понимать и использовать ваш код будут только в том случае, если есть понимание того, что и зачем происходит и как этим пользоваться.***

Пример документации и R

Для нас будет приемлимо написать по-русски, объяснить что происходит, какие есть атрибуты, методы и т.д.



```
Return the First or Last Parts of an Object
Description
Returns the first or last parts of a vector, matrix, table, data frame or function. Since head() and tail() are generic functions, they may also have been extended to other classes.

Usage
head(x, ...)
## Default S3 method:
head(x, n = 6L, ...)

## S3 method for class 'matrix'
head(x, n = 6L, ...) # is exported as head.matrix()
## NB: The methods for 'data.frame' and 'array'  are identical to the 'matrix' one

## S3 method for class 'ftable'
head(x, n = 6L, ...)
## S3 method for class 'function'
head(x, n = 6L, ...)


tail(x, ...)
## Default S3 method:
tail(x, n = 6L, keepnums = FALSE, addrownums, ...)
## S3 method for class 'matrix'
tail(x, n = 6L, keepnums = TRUE, addrownums, ...) # exported as tail.matrix()
## NB: The methods for 'data.frame', 'array', and 'table'
##     are identical to the  'matrix'  one

## S3 method for class 'ftable'
tail(x, n = 6L, keepnums = FALSE, addrownums, ...)
## S3 method for class 'function'
tail(x, n = 6L, ...)
Arguments
x
an object

n
an integer vector of length up to dim(x) (or 1, for non-dimensioned objects). Values specify the indices to be selected in the corresponding dimension (or along the length) of the object. A positive value of n[i] includes the first/last n[i] indices in that dimension, while a negative value excludes the last/first abs(n[i]), including all remaining indices. NA or non-specified values (when length(n) < length(dim(x))) select all indices in that dimension. Must contain at least one non-missing value.

keepnums
in each dimension, if no names in that dimension are present, create them using the indices included in that dimension. Ignored if dim(x) is NULL or its length 1.

addrownums
deprecated - keepnums should be used instead. Taken as the value of keepnums if it is explicitly set when keepnums is not.

...
arguments to be passed to or from other methods.

Details
For vector/array based objects, head() (tail()) returns a subset of the same dimensionality as x, usually of the same class. For historical reasons, by default they select the first (last) 6 indices in the first dimension ("rows") or along the length of a non-dimensioned vector, and the full extent (all indices) in any remaining dimensions. head.matrix() and tail.matrix() are exported.

The default and array(/matrix) methods for head() and tail() are quite general. They will work as is for any class which has a dim() method, a length() method (only required if dim() returns NULL), and a [ method (that accepts the drop argument and can subset in all dimensions in the dimensioned case).

For functions, the lines of the deparsed function are returned as character strings.

When x is an array(/matrix) of dimensionality two and more, tail() will add dimnames similar to how they would appear in a full printing of x for all dimensions k where n[k] is specified and non-missing and dimnames(x)[[k]] (or dimnames(x) itself) is NULL. Specifically, the form of the added dimnames will vary for different dimensions as follows:

k=1 (rows):
"[n,]" (right justified with whitespace padding)

k=2 (columns):
"[,n]" (with no whitespace padding)

k>2 (higher dims):
"n", i.e., the indices as character values

Setting keepnums = FALSE suppresses this behaviour.

As data.frame subsetting (‘indexing’) keeps attributes, so do the head() and tail() methods for data frames.

Value
An object (usually) like x but generally smaller. Hence, for arrays, the result corresponds to x[.., drop=FALSE]. For ftable objects x, a transformed format(x).

Note
For array inputs the output of tail when keepnums is TRUE, any dimnames vectors added for dimensions >2 are the original numeric indices in that dimension as character vectors. This means that, e.g., for 3-dimensional array arr, tail(arr, c(2,2,-1))[ , , 2] and tail(arr, c(2,2,-1))[ , , "2"] may both be valid but have completely different meanings.

Author(s)
Patrick Burns, improved and corrected by R-Core. Negative argument added by Vincent Goulet. Multi-dimension support added by Gabriel Becker.

Examples
Run examples

head(letters)
head(letters, n = -6L)

head(freeny.x, n = 10L)
head(freeny.y)

head(iris3)
head(iris3, c(6L, 2L))
head(iris3, c(6L, -1L, 2L))

tail(letters)
tail(letters, n = -6L)

tail(freeny.x)
## the bottom-right "corner" :
tail(freeny.x, n = c(4, 2))
tail(freeny.y)

tail(iris3)
tail(iris3, c(6L, 2L))
tail(iris3, c(6L, -1L, 2L))

## iris with dimnames stripped
a3d <- iris3 ; dimnames(a3d) <- NULL
tail(a3d, c(6, -1, 2)) # keepnums = TRUE is default here!
tail(a3d, c(6, -1, 2), keepnums = FALSE)

## data frame w/ a (non-standard) attribute:
treeS <- structure(trees, foo = "bar")
(n <- nrow(treeS))
stopifnot(exprs = { # attribute is kept
    identical(htS <- head(treeS), treeS[1:6, ])
    identical(attr(htS, "foo") , "bar")
    identical(tlS <- tail(treeS), treeS[(n-5):n, ])
    ## BUT if I use "useAttrib(.)", this is *not* ok, when n is of length 2:
    ## --- because [i,j]-indexing of data frames *also* drops "other" attributes ..
    identical(tail(treeS, 3:2), treeS[(n-2):n, 2:3] )
})

tail(library) # last lines of function

head(stats::ftable(Titanic))

## 1d-array (with named dim) :
a1 <- array(1:7, 7); names(dim(a1)) <- "O2"
stopifnot(exprs = {
  identical( tail(a1, 10), a1)
  identical( head(a1, 10), a1)
  identical( head(a1, 1), a1 [1 , drop=FALSE] ) # was a1[1] in R <= 3.6.x
  identical( tail(a1, 2), a1[6:7])
  identical( tail(a1, 1), a1 [7 , drop=FALSE] ) # was a1[7] in R <= 3.6.x
})
```

